# Jumping Kodee

This notebook shows an animated Kodee sprite walking on the bottom of the JetBrains IDE window.

It uses a custom Swing component that loads an animated GIF resource and renders it as an animation.

Additionally, it demonstrates how to remove the animation by properly disposing of resources when the Kotlin Notebook kernel is restarted.

In [1]:
%use intellij-platform

IntelliJ Platform integration is loaded

In [2]:
import com.intellij.openapi.Disposable
import com.intellij.openapi.util.Disposer
import com.intellij.openapi.ui.DialogEarthquakeShaker
import com.intellij.openapi.wm.WindowManager
import java.awt.Dimension
import java.awt.Graphics
import java.net.URL
import javax.swing.ImageIcon
import javax.swing.JLabel
import javax.swing.Timer
import kotlin.math.PI
import kotlin.math.cos

class JumpingGifLabel(url: URL, onLand: () -> Unit) : JLabel(ImageIcon(url)), Disposable {
    private var idx = 0
    private val cycle = 17
    private val ascent = 8.0
    private val timer = Timer(40) {
        parent ?: return@Timer
        val (w, h) = with(preferredSize) { width to height }
        val amp = (h / 2.0).coerceAtLeast(ascent)
        val jump = when {
            idx <= ascent -> amp * (1 - cos(PI * (idx / ascent))) / 2
            else -> amp * (1 + cos(PI * ((idx - ascent) / (cycle - 1 - ascent)))) / 2
        }

        setBounds((parent.width - w) / 2, parent.height - h - jump.toInt(), w, h)

        if (idx == cycle - 1) onLand()
        idx = (idx + 1) % cycle
        repaint()
    }

    init {
        preferredSize = Dimension(icon.iconWidth, icon.iconHeight)
    }

    override fun addNotify() {
        super.addNotify()
        timer.start()
    }

    override fun removeNotify() {
        timer.stop()
        super.removeNotify()
    }

    override fun paintComponent(g: Graphics) {
        g.drawImage((icon as ImageIcon).image, 0, 0, width, height, this)
    }

    override fun dispose() = timer.stop()
}

runInEdt {
    val frame = WindowManager.getInstance()
        .getFrame(currentProject())
        ?: error("IDE frame not found")

    val kodee = javaClass
        .getResource("jumping_kodee/kodee-happy.gif")
        ?: error("Kodee not found")

    val label = JumpingGifLabel(kodee) {
        DialogEarthquakeShaker.shake(frame)
    }

    frame.layeredPane.add(label)
    Disposer.register(notebookDisposable) {
        frame.layeredPane.remove(label)
        label.dispose()
    }
}


IntelliJ Platform integration is disposed